## Network Flow Example

We have a network of $m$ directed edges that support $n$ flows, each of which goes over a fixed path in the graph.  We denote the flow values as $f \in \mathbb{R}_+^n$.  The resulting traffic on the $m$ edges is given by $R f$, where $R \in \{0,1\}^{m \times n}$ is the routing matrix, with $R_{ij}=1$ if flow $j$ goes over edge $i$, and 0 otherwise.

The edges have capacity $c \in \mathbb{R}_+^m$, so we have $Rf \leq c$. The objective is to maximize the total utility, which is $U(f) = \sum_i U_i(f_i)$, with

\begin{equation}
U_i(f_i) = \begin{cases}
w_i f_i \quad &\text{if} \quad f^\mathrm{min}_i \leq f_i \leq f^\mathrm{max}_i \\
-\infty \quad &\text{otherwise}
\end{cases}
\end{equation}

To summarize in vector form, we want to solve the following optimization problem:

\begin{equation}
\begin{array}{II}
\text{maximize} \quad &w^T f \\
\text{subject to} \quad &R f \leq c \\
&f^\mathrm{min} \leq f \leq f^\mathrm{max}
\end{array}
\end{equation}

with parameters $w\geq 0$, $R$, $c \geq 0$ and $0 \leq f^\mathrm{min} \leq f^\mathrm{max}$.

Let's define the corresponding CVXPY problem:

In [1]:
import cvxpy as cp
import numpy as np

# define dimensions
n, m = 100, 10

# define variable
f = cp.Variable(n, name='w')

# define parameters
R = cp.Parameter((m, n), name='R')
c = cp.Parameter(m, nonneg=True, name='c')
w = cp.Parameter(n, nonneg=True, name='w')
f_min = cp.Parameter(n, nonneg=True, name='f_min')
f_max = cp.Parameter(n, nonneg=True, name='f_max')

# define objective
objective = cp.Maximize(w@f)

# define constraints
constraints = [R@f <= c, f_min <= f, f <= f_max]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
# TODO

val = problem.solve(solver='OSQP')

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='network_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from network_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('network_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
# TODO

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(solver='OSQP', eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)